# Index Stock Performance

In [40]:
!pip install requests pandas yfinance investpy lxml beautifulsoup4 tabulate

In [30]:
import yfinance as yf
import pandas as pd
from datetime import datetime

## Fetch index components

In [86]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor

def fetch_components_stockanalysis(index):
    base_url = "https://stockanalysis.com/etf/"
    index_map = {
        'QQQ': 'qqq',
        'SPX': 'spy',
        'DOW': 'dia',
        'IWM': 'iwm'
    }
    if index.upper() not in index_map:
        raise ValueError("Index must be either 'QQQ', 'SPX', 'DOW', or 'IWM'")
    
    url = f"{base_url}{index_map[index.upper()]}/holdings/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find('table')
    
    data = []
    headers = []
    
    for i, row in enumerate(table.find_all('tr')):
        cols = [ele.text.strip() for ele in row.find_all('td')]
        if i == 0:
            headers = [ele.text.strip() for ele in row.find_all('th')]
        else:
            if len(cols) > 1:  # Ensure there is data in the row
                data.append(cols)
    
    df = pd.DataFrame(data, columns=headers)
    return df

def print_all_company_details_compact(df):
    # Print DataFrame columns for debugging
    print("DataFrame columns:", df.columns)
    
    # Convert 'Weight' or 'Portfolio %' column to numeric if it exists
    weight_column = 'Weight' if 'Weight' in df.columns else 'Portfolio %' if 'Portfolio %' in df.columns else None
    if weight_column:
        df[weight_column] = df[weight_column].str.replace('%', '').astype(float)
        df = df.sort_values(by=weight_column, ascending=False)
    
    # Get the maximum length of each column for formatting
    col_widths = [max(df[col].astype(str).map(len).max(), len(col)) for col in df.columns]
    format_str = ' | '.join([f'{{:<{width}}}' for width in col_widths])
    
    print(format_str.format(*df.columns))
    print('-' * (sum(col_widths) + 3 * (len(col_widths) - 1)))
    
    for index, row in df.iterrows():
        print(format_str.format(*row))

# Example usage:
# Parameters
index = 'QQQ'  # Change to 'SPX' for S&P 500 or 'DOW' for Dow Jones

# Fetch data from StockAnalysis
df = fetch_components_stockanalysis(index)

# Print details of all companies in a compact format
print_all_company_details_compact(df)

DataFrame columns: Index(['No.', 'Symbol', 'Name', '% Weight', 'Shares'], dtype='object')
No. | Symbol | Name                                | % Weight | Shares     
---------------------------------------------------------------------------
1   | MSFT   | Microsoft Corporation               | 8.78%    | 55,597,683 
2   | AAPL   | Apple Inc                           | 8.59%    | 114,707,162
3   | NVDA   | NVIDIA Corporation                  | 7.90%    | 184,008,984
4   | AMZN   | Amazon.com, Inc.                    | 5.31%    | 77,847,189 
5   | AVGO   | Broadcom Inc.                       | 5.08%    | 8,953,887  
6   | META   | Meta Platforms, Inc.                | 4.55%    | 26,079,909 
7   | GOOGL  | Alphabet Inc.                       | 2.78%    | 43,940,628 
8   | GOOG   | Alphabet Inc.                       | 2.68%    | 42,018,261 
9   | TSLA   | Tesla, Inc.                         | 2.62%    | 36,111,203 
10  | COST   | Costco Wholesale Corporation        | 2.50%    | 8,569,054 

KeyError: 'Company'

In [88]:
# Function to fetch components using Alpha Vantage
def fetch_components_alphavantage(index, api_key):
    index_map = {
        'QQQ': 'NASDAQ100',
        'SPX': 'SP500',
        'DOW': 'DOWJONES',
        'IWM': 'RUSSELL2000'
    }
    
    if index.upper() not in index_map:
        raise ValueError("Index must be either 'QQQ', 'SPX', 'DOW', or 'IWM'")
    
    url = f"https://www.alphavantage.co/query?function=SECTOR&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    
    if "Rank A: Real-Time Performance" not in data:
        raise ValueError("Invalid response from API or API rate limit exceeded")
    
    components_data = []
    symbols = data["Rank A: Real-Time Performance"]
    
    for symbol in symbols:
        components_data.append({'Symbol': symbol, 'Name': yf.Ticker(symbol).info.get('shortName', 'N/A')})
    
    df = pd.DataFrame(components_data)
    return df

def print_all_company_details_compact(df):
    col_widths = [max(df[col].astype(str).map(len).max(), len(col)) for col in df.columns]
    format_str = ' | '.join([f'{{:<{width}}}' for width in col_widths])
    
    print(format_str.format(*df.columns))
    print('-' * (sum(col_widths) + 3 * (len(col_widths) - 1)))
    
    for index, row in df.iterrows():
        print(format_str.format(*row))


# Example usage:
# Parameters
api_key = 'UUULX0VXMWTKN8JY'  # Replace with your Alpha Vantage API key
index = 'SPX'  # Change to 'QQQ' for Nasdaq-100, 'DOW' for Dow Jones, 'IWM' for Russell 2000

# Fetch data from Alpha Vantage
df = fetch_components_alphavantage(index, api_key)

# Print details of all companies in a compact format
print_all_company_details_compact(df)

Error fetching data for ABNB: index 0 is out of bounds for axis 0 with size 0
Top 20 Stocks:
TSLA (Tesla, Inc.): 16.9%
AAPL (Apple Inc): 4.6%
AMZN (Amazon.com, Inc.): 3.5%
AVGO (Broadcom Inc.): 3.2%
CEG (Constellation Energy Corporation): 2.9%
CDNS (Cadence Design Systems, Inc.): 2.9%
MSFT (Microsoft Corporation): 2.8%
ASML (ASML Holding N.V.): 2.5%
SNPS (Synopsys, Inc.): 2.5%
ADBE (Adobe Inc.): 2.2%
TMUS (T-Mobile US, Inc.): 2.2%
AMAT (Applied Materials, Inc.): 2.1%
TXN (Texas Instruments Incorporated): 2.0%
KLAC (KLA Corporation): 1.8%
GOOG (Alphabet Inc.): 1.7%
GOOGL (Alphabet Inc.): 1.7%
AMD (Advanced Micro Devices, Inc.): 1.3%
COST (Costco Wholesale Corporation): 1.1%
NXPI (NXP Semiconductors N.V.): 1.1%
VRTX (Vertex Pharmaceuticals Incorporated): 1.1%

Bottom 20 Stocks:
CRWD (CrowdStrike Holdings, Inc.): 0.5%
PANW (Palo Alto Networks, Inc.): 0.4%
INTC (Intel Corporation): 0.3%
ADI (Analog Devices, Inc.): -0.0%
HON (Honeywell International Inc.): -0.1%
INTU (Intuit Inc.): -0.2%
GI

In [85]:
def calculate_performance(symbol, start_date, end_date):
    stock = yf.Ticker(symbol)
    hist = stock.history(start=start_date, end=end_date)
    
    if len(hist) == 0:
        return None
    
    start_price = hist['Close'].iloc[0]
    end_price = hist['Close'].iloc[-1]
    performance = (end_price - start_price) / start_price
    return performance

def get_performance_data(symbol, company, start_date, end_date):
    try:
        performance = calculate_performance(symbol, start_date, end_date)
        if performance is not None:
            return (symbol, company, performance)
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
    return None

def fetch_all_performance_data(df, start_date, end_date):
    performance_data = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(get_performance_data, row['Symbol'], row['Name'], start_date, end_date) for index, row in df.iterrows()]
        for future in futures:
            result = future.result()
            if result:
                performance_data.append(result)
    
    performance_data.sort(key=lambda x: x[2], reverse=True)
    return performance_data

def display_top_bottom_stocks(df, period, top_n, bottom_n, start_year=None, end_year=None):
    end_date = datetime.now()
    
    if start_year and end_year:
        start_date = datetime(start_year, 1, 1)
        end_date = datetime(end_year, 12, 31)
    else:
        if period == 'ytd':
            start_date = datetime(end_date.year, 1, 1)
        else:
            delta = {
                '1d': 1,
                '5d': 5,
                '1mo': 30,
                '6mo': 182,
                '1y': 365,
                '2y': 365*2,
                '3y': 365*3,
                '5y': 365*5,
                '10y': 365*10,
                '20y': 365*20
            }[period]
            start_date = end_date - timedelta(days=delta)
    
    performance_data = fetch_all_performance_data(df, start_date, end_date)
    
    print(f"Top {top_n} Stocks:")
    for symbol, company, performance in performance_data[:top_n]:
        print(f"{symbol} ({company}): {performance*100:.1f}%")
    
    print(f"\nBottom {bottom_n} Stocks:")
    for symbol, company, performance in performance_data[-bottom_n:]:
        print(f"{symbol} ({company}): {performance*100:.1f}%")

# Example usage:
# Parameters
index = 'IWM'  # Change to 'SPX' for S&P 500 or 'DOW' for Dow Jones
period = '5d'  # Options: 'ytd', '1d', '5d', '1mo', '6mo', '1y', '2y', '3y', '5y', '10y', '20y'
top_n = 20
bottom_n = 20
start_year = None  # Example: 2004
end_year = None  # Example: 2024

# Fetch data from StockAnalysis
df = fetch_components_stockanalysis(index)

# Display top and bottom performing stocks
display_top_bottom_stocks(df, period, top_n, bottom_n, start_year, end_year)

$XTSLA: possibly delisted; No timezone found


Top 20 Stocks:
MARA (Marathon Digital Holdings, Inc.): 11.4%
MOD (Modine Manufacturing Company): 5.2%
FTAI (FTAI Aviation Ltd.): 4.6%
CVLT (Commvault Systems, Inc.): 3.4%
ANF (Abercrombie & Fitch Co.): 3.1%
SPSC (SPS Commerce, Inc.): 2.4%
BPMC (Blueprint Medicines Corporation): 2.4%
RMBS (Rambus Inc.): 2.0%
TRNO (Terreno Realty Corporation): 2.0%
ENSG (The Ensign Group, Inc.): 1.8%
JXN (Jackson Financial Inc.): 1.6%
CYTK (Cytokinetics, Incorporated): 1.2%
NSIT (Insight Enterprises, Inc.): 1.0%
SSB (SouthState Corporation): 1.0%
PCVX (Vaxcyte, Inc.): 0.7%
ESNT (Essent Group Ltd.): 0.7%
QLYS (Qualys, Inc.): 0.7%
GKOS (Glaukos Corporation): 0.5%
ONB (Old National Bancorp): 0.5%
ALTR (Altair Engineering Inc.): 0.3%

Bottom 20 Stocks:
ETRN (Equitrans Midstream Corporation): -0.6%
FLR (Fluor Corporation): -0.8%
SFM (Sprouts Farmers Market, Inc.): -0.8%
COOP (Mr. Cooper Group Inc.): -0.9%
AIT (Applied Industrial Technologies, Inc.): -1.1%
FN (Fabrinet): -1.3%
GTLS (Chart Industries, Inc.): -1